In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as pt
from dash import Dash, dcc, html, Input, Output

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('all_fuels_data.csv')
df['date'] = pd.to_datetime(df['date'])

In [3]:
df.head()

,ticker,commodity,date,open,high,low,close,volume
0,CL=F,Crude Oil,2000-08-23,31.950001,32.799999,31.950001,32.049999,79385
1,CL=F,Crude Oil,2000-08-24,31.900000,32.240002,31.400000,31.629999,72978
2,CL=F,Crude Oil,2000-08-25,31.700001,32.099998,31.320000,32.049999,44601
3,CL=F,Crude Oil,2000-08-28,32.040001,32.919998,31.860001,32.869999,46770
4,CL=F,Crude Oil,2000-08-29,32.820000,33.029999,32.560001,32.720001,49131


In [4]:
df.describe()

,date,open,high,low,close,volume
count,27155,27155.000000,27155.000000,27155.000000,27155.000000,2.715500e+04
mean,2012-10-02 09:10:42.312649472,27.064888,27.454546,26.651262,27.064125,1.055993e+05
min,2000-08-23 00:00:00,-14.000000,0.507000,-40.320000,-37.630001,0.000000e+00
25%,2007-06-30 12:00:00,2.008450,2.039750,1.976900,2.007700,2.611850e+04
50%,2012-12-06 00:00:00,3.425000,3.505500,3.346000,3.428000,4.860800e+04
75%,2018-05-08 00:00:00,53.599998,54.410000,52.719999,53.625000,1.129145e+05
max,2023-09-29 00:00:00,146.080002,147.429993,144.270004,146.080002,2.288230e+06
std,NaN,35.976657,36.430093,35.491529,35.981681,1.493846e+05


In [5]:
df.commodity.unique().tolist()

['Crude Oil', 'Heating Oil', 'Natural Gas', 'RBOB Gasoline', 'Brent Crude Oil']

In [6]:
def trendline_plot(data, order=1):
    coeffs = np.polyfit(data.index.values, list(data), order)
    slope = coeffs[-2]
    intercept = coeffs[-1]
    return [slope * i + intercept for i in data.index.values]

def seasonality_plot(data):
    return data - trendline_plot(data)

In [7]:
app = Dash(__name__)

app.layout = html.Div([
    html.H3("Commodity Price Dashboard"),
    dcc.Graph(id='time-series-graph'),
    html.P("Select stock:"),
    dcc.Dropdown(
        id='commodity-dropdown',
        options=[
            {'label': 'Crude Oil', 'value': 'Crude Oil'},
            {'label': 'Heating Oil', 'value': 'Heating Oil'},
            {'label': 'Natural Gas', 'value': 'Natural Gas'},
            {'label': 'RBOB Gasoline', 'value': 'RBOB Gasoline'},
            {'label': 'Brent Crude Oil', 'value': 'Brent Crude Oil'},
        ],
        value='Crude Oil'
    ),
])

@app.callback(
    Output('time-series-graph', 'figure'),
    Input('commodity-dropdown', 'value')
)
def update_graph(selected_commodity):
    filtered_df = df[df['commodity'] == selected_commodity]
    fig = pt.line(filtered_df, x='date', y='close',color='commodity',title=f'Time series plot of {selected_commodity}')
    fig.add_scatter(x=filtered_df['date'], y=trendline_plot(filtered_df['close'], order=1), mode='lines', name='trendline')
    fig.add_scatter(x=filtered_df['date'], y=seasonality_plot(filtered_df['close']), mode='lines', name='seasonality')
    fig.update_layout(showlegend=True)
    return fig

app.run_server(debug=True)